# Forcasting Package Tutorial

This notebook demonstrates the basic usage of the forcasting_pkg for financial forecasting and technical analysis.

In [ ]:
# Install the package if not already installed
# !pip install forcasting-pkg[full]

In [ ]:
# Import required modules
from forcasting_pkg import ForecastingEngine, TechnicalAnalyzer
from forcasting_pkg.data import get_historical_data
from forcasting_pkg.visualization import plot_forecast, plot_technical_indicators
from forcasting_pkg.models import DataType

import pandas as pd
import matplotlib.pyplot as plt

print("Forcasting Package loaded successfully!")

## 1. Data Acquisition

Let's start by fetching some historical stock data.

In [ ]:
# Get historical data for Apple stock
symbol = "AAPL"
data = get_historical_data(symbol, days=90, data_type=DataType.STOCK)

print(f"Loaded {len(data)} data points for {symbol}")
print(f"Date range: {data[0].date.strftime('%Y-%m-%d')} to {data[-1].date.strftime('%Y-%m-%d')}")
print(f"Price range: ${min(d.close for d in data):.2f} - ${max(d.close for d in data):.2f}")

## 2. Basic Forecasting

Now let's generate some forecasts using different models.

In [ ]:
# Create forecasting engine
engine = ForecastingEngine()

# Generate ARIMA forecast
arima_forecast = engine.forecast(data, "arima", symbol, days=30)

print(f"ARIMA Forecast:")
print(f"  Model: {arima_forecast.model_type}")
print(f"  Accuracy: {arima_forecast.model_accuracy:.2%}" if arima_forecast.model_accuracy else "  Accuracy: N/A")
print(f"  30-day forecast: ${arima_forecast.forecast_points[0].predicted_value:.2f} (first day)")

In [ ]:
# Compare multiple models
comparison = engine.compare_models(data, symbol, days=14)

print("Model Comparison (14-day forecast):")
print("-" * 40)

for model_name, forecast in comparison.items():
    accuracy = forecast.model_accuracy if forecast.model_accuracy else 0
    first_pred = forecast.forecast_points[0].predicted_value if forecast.forecast_points else 0
    print(f"{model_name:20} | Accuracy: {accuracy:>6.2%} | First pred: ${first_pred:>7.2f}")

## 3. Technical Analysis

Let's analyze the technical indicators for our stock.

In [ ]:
# Perform technical analysis
analyzer = TechnicalAnalyzer()
analysis = analyzer.analyze(data, symbol)

print(f"Technical Analysis for {symbol}:")
print(f"  Signal Strength: {analysis.signal_strength}")
print(f"  Confidence: {analysis.confidence:.2%}" if analysis.confidence else "  Confidence: N/A")

# Display key indicators
indicators = analysis.technical_indicators
print("\nKey Indicators:")

if indicators.rsi:
    print(f"  RSI: {indicators.rsi:.2f}")
if indicators.macd:
    print(f"  MACD: {indicators.macd:.4f}")
if indicators.moving_average_20:
    print(f"  20-day MA: ${indicators.moving_average_20:.2f}")
if indicators.moving_average_50:
    print(f"  50-day MA: ${indicators.moving_average_50:.2f}")

# Display recommendations
if analysis.recommendations:
    print("\nRecommendations:")
    for rec in analysis.recommendations:
        print(f"  • {rec}")

## 4. Visualization

Let's create some plots to visualize our analysis.

In [ ]:
# Plot forecast
fig = plot_forecast(data, arima_forecast, symbol, interactive=False)
if fig:
    plt.show()
else:
    print("Visualization not available (missing matplotlib or plotly)")

In [ ]:
# Plot technical indicators
fig = plot_technical_indicators(data, analysis.technical_indicators, symbol, interactive=False)
if fig:
    plt.show()
else:
    print("Technical analysis plot not available")

## 5. Ensemble Forecasting

Let's create an ensemble forecast that combines multiple models.

In [ ]:
# Create ensemble forecast
ensemble_forecast = engine.create_ensemble(
    data, 
    symbol, 
    models=["arima", "linear", "moving_average"],
    weights=[0.5, 0.3, 0.2],  # Higher weight for ARIMA
    days=21
)

print(f"Ensemble Forecast:")
print(f"  Model: {ensemble_forecast.model_type}")
print(f"  Accuracy: {ensemble_forecast.model_accuracy:.2%}" if ensemble_forecast.model_accuracy else "  Accuracy: N/A")
print(f"  21-day forecast generated")

# Show some predictions
print("\nSample predictions:")
for i in [0, 6, 13, 20]:  # Weekly intervals
    if i < len(ensemble_forecast.forecast_points):
        point = ensemble_forecast.forecast_points[i]
        print(f"  Day {i+1:2d}: ${point.predicted_value:.2f}")

## 6. Cryptocurrency Analysis

The package also supports cryptocurrency analysis.

In [ ]:
# Get Bitcoin data
crypto_data = get_historical_data("bitcoin", days=60, data_type=DataType.CRYPTO)

if crypto_data:
    print(f"Loaded {len(crypto_data)} data points for Bitcoin")
    
    # Generate crypto forecast
    btc_forecast = engine.forecast(crypto_data, "arima", "BTC", days=7)
    
    print(f"\nBitcoin 7-day forecast:")
    print(f"  Model: {btc_forecast.model_type}")
    
    # Show predictions
    for i, point in enumerate(btc_forecast.forecast_points[:3]):
        print(f"  Day {i+1}: ${point.predicted_value:,.2f}")
else:
    print("Bitcoin data not available (using mock data)")

## 7. Export Results

Finally, let's export our forecast results.

In [ ]:
# Export forecast to dictionary
forecast_dict = arima_forecast.dict()

print("Forecast exported to dictionary:")
print(f"  Symbol: {forecast_dict['symbol']}")
print(f"  Model: {forecast_dict['model_type']}")
print(f"  Forecast points: {len(forecast_dict['forecast_points'])}")

# Convert to pandas DataFrame for analysis
forecast_df = pd.DataFrame([
    {
        'date': point['date'],
        'prediction': point['predicted_value'],
        'lower_bound': point['confidence_interval_lower'],
        'upper_bound': point['confidence_interval_upper']
    } for point in forecast_dict['forecast_points']
])

print("\nForecast DataFrame:")
print(forecast_df.head())

## Conclusion

This notebook demonstrated the basic capabilities of the forcasting_pkg:

- **Data Acquisition**: Fetching historical market data
- **Forecasting**: ARIMA, Linear, and Moving Average models
- **Technical Analysis**: RSI, MACD, and other indicators
- **Visualization**: Plotting forecasts and technical indicators
- **Ensemble Methods**: Combining multiple models
- **Cryptocurrency Support**: Analysis of crypto markets
- **Data Export**: Converting results to various formats

For more advanced features, check out the advanced_usage.py example and the package documentation.